In [2]:
#!/usr/bin/env python
# coding: utf-8
# In[14]:
import cv2
import numpy as np
import copy
from matplotlib import pyplot as plt
"""
#メモ
①黒板(対象物)をとる
②授業動画をとる
③①と②の画像一枚一枚を見比べる(特徴の判定）
④特徴から画像を切り出す
⑤切り出した画像を透視返還(台形返還）
⑥ノートにする
​
普通に特徴点をとったほうがいいかも
しいて言えば二値化すれば...
↑
今のところcannyと二値化しか試してないので他のedge検知のアルゴリズムを試してみるべき？
あとはフーリエ変換かな。。。。
正面の傾きには強いが横からなど角度がつくと極端にダメになる
機械学習を用いた対応が必要になるのではないか？？？？
2019/05/11記入
"""
std = "img/output/"
stdin = "img/input/"
testcase1 =["IMG_78645","IMG_7864","IMG_7865","IMG_7866","IMG_7867","IMG_7868","IMG_7869","IMG_7870","IMG_7871",
            "IMG_7872","IMG_7873","IMG_7874","IMG_7875"] 
now = testcase1
TARGET = 0
IMAGE = []
img,image,gray,rgba=[],[],[],[]
timg,tgray,trgba=[],[],[]
def setup(string):
    string+=".JPG"
    global img
    global IMAGE
    img.append(cv2.imread(stdin+string))
    IMAGE.append(copy.deepcopy(img))
    global gray 
    gray.append(cv2.imread(stdin+string,0))
    global rgba 
    rgba.append(cv2.imread(stdin+string,-1))
def setuptmp(string):
    string+=".png"
    global timg 
    timg.append(cv2.imread(stdin+string))
    global tgray 
    tgray.append(cv2.imread(stdin+string,0))
    global trgba 
    trgba.append(cv2.imread(stdin+string,-1))
for i in range(0,len(now)):
    setup(now[i])
    #cv2.imwrite(stdin+now[i]+".png",img[i])
    now[i]+=".jpg"
print("fin")

fin


In [ ]:
#特徴点検出AkAZE法with 2値化
#目標画像の加工
# 閾値
t = 200
# 方法2値化
ret,th = [],[]
for i in range(0,len(now)):
    ref,tmp = cv2.threshold(gray[i],t,255,CV_THRESH_TOZERO_INV)
    tmp = cv2.bitwise_not(tmp)
    ref,tmp = cv2.threshold(tmp, 0, 255, cv2.THRESH_BINARY| CV_THRESH_OTSU);
    ret.append(ref) 
    th.append(tmp)
    # 結果を出力
    cv2.imwrite(std+"th2"+now[i], tmp)

akaze = cv2.AKAZE_create()
#特徴点比較AKAZE法with 2値化
kp, des = [],[]
for i in th:
    ref,tmp= akaze.detectAndCompute(i, None)
    kp.append(ref)
    des.append(tmp)

bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
for i in range(1,len(now)):
    matches = bf.match(des[TARGET], des[i])
    matches = sorted(matches, key = lambda x:x.distance)
    img_compare = cv2.drawMatches(img[TARGET], kp[TARGET], img[i], kp[i], matches[:10], None, flags=2)
    cv2.imwrite(std+"thcompare"+now[TARGET]+"&"+now[i],img_compare)

bf = cv2.BFMatcher()
for i in range(1,len(now)):
    matches = bf.knnMatch(des[TARGET], des[i], k=2)
    good = []
    for m,n in matches:
        if m.distance < 0.5*n.distance:
            good.append([m])
    img_compare2 = cv2.drawMatchesKnn(img[TARGET], kp[TARGET], img[i], kp[i], good, None, flags=2)
    cv2.imwrite(std+"thcompare2"+now[TARGET]+"&"+now[i],img_compare2)

def alignImages(img1, img2,i,
                max_pts=500, good_match_rate=0.15, min_match=10):
    # https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_feature2d/py_feature_homography/py_feature_homography.html#feature-homography

    # [1] ORBを用いて特徴量を検出する
    # Initiate ORB detector
    detector = cv2.ORB_create(max_pts)
    # find the keypoints and descriptors with SIFT
    kp1, des1 = detector.detectAndCompute(
        cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY), None
    )
    kp2, des2 = detector.detectAndCompute(
        cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY), None
    )
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

    # [2] 検出した特徴量の比較をしてマッチングをする
    # Match descriptors.
    matches = bf.match(des1, des2)

    # Sort them in the order of their distance.
    matches = sorted(matches, key=lambda x: x.distance)
    good = matches[:int(len(matches) * good_match_rate)]

    # [3] 十分な特徴量が集まったらそれを使って入力画像を変形する
    if len(good) > min_match:
        src_pts = np.float32(
            [kp1[m.queryIdx].pt for m in good]).reshape(-1, 1, 2)
        dst_pts = np.float32(
            [kp2[m.trainIdx].pt for m in good]).reshape(-1, 1, 2)
        # Find homography
        h, mask = cv2.findHomography(dst_pts, src_pts, cv2.RANSAC)

        cv2.imwrite(std+"changed"+now[i], 
                    cv2.drawMatches(img1, kp1, img2, kp2, matches, None, flags=2))

        # Use homography
        height, width, channels = img1.shape
        dst_img = cv2.warpPerspective(img2, h, (width, height))
        return dst_img
    else:
        return img1, np.zeros((3, 3))

for i in range(1,len(now)): 
    change = alignImages(img[TARGET],img[i],i)
    #第一引数に目標画像、第二引数に対象画像
    cv2.imwrite(std+"changed"+now[i],change)
print("fin")


In [4]:
#特徴点検出AkAZE法with 2値化
#目標画像の加工
# 閾値
t = 200
# 方法2値化
ret,th = [],[]
for i in range(0,len(now)):
    ref,tmp = cv2.threshold(gray[i],t,255,cv2.THRESH_TOZERO_INV)
    tmp = cv2.bitwise_not(tmp)
    ref,tmp = cv2.threshold(tmp, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU);
    ret.append(ref) 
    th.append(tmp)
    # 結果を出力
    cv2.imwrite(std+"th2"+now[i], tmp)

akaze = cv2.AKAZE_create()
#特徴点比較AKAZE法with 2値化
kp, des = [],[]
for i in th:
    ref,tmp= akaze.detectAndCompute(i, None)
    kp.append(ref)
    des.append(tmp)

bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
for i in range(1,len(now)):
    matches = bf.match(des[TARGET], des[i])
    matches = sorted(matches, key = lambda x:x.distance)
    img_compare = cv2.drawMatches(img[TARGET], kp[TARGET], img[i], kp[i], matches[:10], None, flags=2)
    cv2.imwrite(std+"thcompare"+now[TARGET]+"&"+now[i],img_compare)

bf = cv2.BFMatcher()
for i in range(1,len(now)):
    matches = bf.knnMatch(des[TARGET], des[i], k=2)
    good = []
    for m,n in matches:
        if m.distance < 0.5*n.distance:
            good.append([m])
    img_compare2 = cv2.drawMatchesKnn(img[TARGET], kp[TARGET], img[i], kp[i], good, None, flags=2)
    cv2.imwrite(std+"thcompare2"+now[TARGET]+"&"+now[i],img_compare2)

def alignImages(img1, img2,i,
                max_pts=500, good_match_rate=0.15, min_match=10):
    # https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_feature2d/py_feature_homography/py_feature_homography.html#feature-homography

    # [1] ORBを用いて特徴量を検出する
    # Initiate ORB detector
    detector = cv2.ORB_create(max_pts)
    # find the keypoints and descriptors with SIFT
    kp1, des1 = detector.detectAndCompute(
        cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY), None
    )
    kp2, des2 = detector.detectAndCompute(
        cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY), None
    )
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

    # [2] 検出した特徴量の比較をしてマッチングをする
    # Match descriptors.
    matches = bf.match(des1, des2)

    # Sort them in the order of their distance.
    matches = sorted(matches, key=lambda x: x.distance)
    good = matches[:int(len(matches) * good_match_rate)]

    # [3] 十分な特徴量が集まったらそれを使って入力画像を変形する
    if len(good) > min_match:
        src_pts = np.float32(
            [kp1[m.queryIdx].pt for m in good]).reshape(-1, 1, 2)
        dst_pts = np.float32(
            [kp2[m.trainIdx].pt for m in good]).reshape(-1, 1, 2)
        # Find homography
        h, mask = cv2.findHomography(dst_pts, src_pts, cv2.RANSAC)

        cv2.imwrite(std+"changed"+now[i], 
                    cv2.drawMatches(img1, kp1, img2, kp2, matches, None, flags=2))

        # Use homography
        height, width, channels = img1.shape
        dst_img = cv2.warpPerspective(img2, h, (width, height))
        return dst_img
    else:
        return img1, np.zeros((3, 3))

for i in range(1,len(now)): 
    change = alignImages(img[TARGET],img[i],i)
    #第一引数に目標画像、第二引数に対象画像
    cv2.imwrite(std+"changed"+now[i],change)
print("fin")

error: OpenCV(4.1.0) C:\projects\opencv-python\opencv\modules\features2d\src\akaze.cpp:172: error: (-215:Assertion failed) ! image.empty() in function 'cv::AKAZE_Impl::detectAndCompute'


In [5]:
#特徴点検出AkAZE法
akaze = cv2.AKAZE_create()
#特徴点比較AKAZE法
kp, des = [],[]
for i in gray:
    ref,tmp= akaze.detectAndCompute(i, None)
    kp.append(ref)
    des.append(tmp)

bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
for i in range(1,len(now)):
    matches = bf.match(des[TARGET], des[i])
    matches = sorted(matches, key = lambda x:x.distance)
    img_compare = cv2.drawMatches(img[TARGET], kp[TARGET], img[i], kp[i], matches[:10], None, flags=2)
    cv2.imwrite(std+"compare"+now[0]+"&"+now[i],img_compare)

bf = cv2.BFMatcher()
for i in range(1,len(now)):
    matches = bf.knnMatch(des[TARGET], des[i], k=2)
    good = []
    for m,n in matches:
        if m.distance < 0.5*n.distance:
            good.append([m])
    img_compare2 = cv2.drawMatchesKnn(img[TARGET], kp[TARGET], img[i], kp[i], good, None, flags=2)
    cv2.imwrite(std+"compare2"+now[TARGET]+"&"+now[i],img_compare2)

def alignImages(img1, img2,i,
                max_pts=500, good_match_rate=0.15, min_match=10):
    # https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_feature2d/py_feature_homography/py_feature_homography.html#feature-homography

    # [1] ORBを用いて特徴量を検出する
    # Initiate ORB detector
    detector = cv2.ORB_create(max_pts)
    # find the keypoints and descriptors with SIFT
    kp1, des1 = detector.detectAndCompute(
        cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY), None
    )
    kp2, des2 = detector.detectAndCompute(
        cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY), None
    )
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

    # [2] 検出した特徴量の比較をしてマッチングをする
    # Match descriptors.
    matches = bf.match(des1, des2)

    # Sort them in the order of their distance.
    matches = sorted(matches, key=lambda x: x.distance)
    good = matches[:int(len(matches) * good_match_rate)]

    # [3] 十分な特徴量が集まったらそれを使って入力画像を変形する
    if len(good) > min_match:
        src_pts = np.float32(
            [kp1[m.queryIdx].pt for m in good]).reshape(-1, 1, 2)
        dst_pts = np.float32(
            [kp2[m.trainIdx].pt for m in good]).reshape(-1, 1, 2)
        # Find homography
        h, mask = cv2.findHomography(dst_pts, src_pts, cv2.RANSAC)

        cv2.imwrite(std+"gchanged"+now[i], 
                    cv2.drawMatches(img1, kp1, img2, kp2, matches, None, flags=2))

        # Use homography
        height, width, channels = img1.shape
        dst_img = cv2.warpPerspective(img2, h, (width, height))
        return dst_img
    else:
        return img1, np.zeros((3, 3))

for i in range(1,len(now)): 
    change = alignImages(img[TARGET],img[i],i)
    #第一引数に目標画像、第二引数に対象画像
    cv2.imwrite(std+"gchanged"+now[i],change)
print("fin")

SyntaxError: invalid character in identifier (<ipython-input-5-1e01df84d58b>, line 9)